In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

## Code which scrapes the recently reviews (only 100 pages - 1,400 reviews)

In [2]:

def ratebeer_scraper(page_start=1, page_end=2, url_set=None):
    
    beer_name = []
    beer_rating = []
    brewery_name = []
    brewery_location = []
    review_content = []
    reviewer_name = []
 
    if url_set == None:
        print("Need url to run")
        return

    for x in tqdm(range(page_start, page_end + 1)):
        try:
            url_format = url_set.format(x)
            r = requests.get(url_format)
            soup = BeautifulSoup(r.text, 'html.parser')
            table = soup.find('table', attrs={'class': 'table'})
            table_rows = table.find_all('tr')

            for rows in table_rows:

            # Scrape Beer Name    
                try:
                    beer_name.append(rows.find('a', attrs={'style':'font-size:20px; font-weight:bold;'}).text)
                except:
                    beer_name.append(None)
            # Scrape Review Rating  
                try:
                    beer_rating.append(rows.find('span', attrs={'class':'uas'}).text)
                except:
                    beer_rating.append(None)
            # Scrape Brewery Name  
                try:
                    brewery_div = rows.find('div')
                    brewery_name.append(brewery_div.a.text)
                except:
                    brewery_name.append(None)
            # Scrape Brewery Location
                try:
                    brewery_location.append(rows.find('span', attrs={'class':'small'}).text.strip())
                except:
                    brewery_location.append(None)
            # Scrape Reviewer Name
                try:
                    reviewer_name.append(rows.find('span', attrs={'class':'pull-right'}).text.strip())
                except:
                    reviewer_name.append(None)
            # Scrape content
                try:
                    review_content.append(rows.find('div', attrs={'style':"color:#666;"}).text.strip())
                except:
                    review_content.append(None)
        except:
            pass

    beerz = pd.DataFrame({'beer_name': beer_name,
                         'beer_rating': beer_rating,
                        'brewery_name': brewery_name,
                         'brewery_location': brewery_location,
                        'review_content': review_content,
                        'reviewer_name': reviewer_name})
    return beerz



In [4]:
# # Scraping most recent reviews
# url = 'https://www.ratebeer.com/beer-ratings/0/{}/'
# df_recent = ratebeer_scraper(0,100,url)

# Scraping new beers
url = 'https://www.ratebeer.com/beer-ratings/5/{}/'
df_new = ratebeer_scraper(0,1,url)

# Scraping fave beers
url = 'https://www.ratebeer.com/beer-ratings/4/{}/'
df_fave = ratebeer_scraper(0,1,url)

# Scraping rave beers
url = 'https://www.ratebeer.com/beer-ratings/1/{}/'
df_rave = ratebeer_scraper(0,1,url)

# Scraping rant beers
url = 'https://www.ratebeer.com/beer-ratings/2/{}/'
df_rant = ratebeer_scraper(0,1,url)

# Scraping anomalous beers
url = 'https://www.ratebeer.com/beer-ratings/3/{}/'
df_anom = ratebeer_scraper(0,1,url)

In [67]:
# getting rid of non-ascii characters from the reviews as it breaks csv
df_rave.review_content = df_rave.review_content.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))

In [5]:
# read current data from csv (there is a backup at all_reviews_backup.csv)
# df = pd.read_csv('all_reviews.csv')

In [25]:
# concatenating all of the scrapes together with the old df
df = pd.concat([df, df_recent, df_new, df_fave, df_rave, df_rant, df_anom])
print(len(df))
df.drop_duplicates(inplace=True)
print(len(df))
df.beer_name = df.beer_name.apply(lambda x: x.strip())



14506
12841


In [26]:
df

,beer_name,beer_rating,brewery_location,brewery_name,review_content,reviewer_name
0,St. Peter's Ruby Red Ale,3.3,"Bungay, Suffolk",St. Peter's Brewery,"Ruby color, offwhite coam. Aroma and taste: fl...",alex_leit
1,Shepherd Neame Spitfire (Bottle),2.2,"Faversham, Kent",Shepherd Neame,Disappointing show from the lads at ‘Britain’s...,shivermetimber.gray
2,Arbor Rocketman,3.8,"Bristol, Gloucestershire",Arbor Ales,"Puszka. Mętne o żółtym kolorze. Piana biała,ob...",MarcinG
3,Shepherd Neame India Pale Ale (Bottle),3.4,"Faversham, Kent",Shepherd Neame,Bottle. Amber with bronze color and white foam...,alex_leit
4,Shepherd Neame Christmas Ale,3.3,"Faversham, Kent",Shepherd Neame,Bottle. Aroma: English hops and malts with tof...,alex_leit
...,...,...,...,...,...,...
10,Grimm Mosaic Pop!,1.6,-1.9,Grimm Artisanal Ales,"660ml bottle (Pothouse, Tallinn) (3:2:3:2:6=1....",FatPhil
11,Bone Machine Garden of Death,1.9,-1.6,Bone Machine Brew Co,Can: Poured a mucky orange brown with huge whi...,SHIG
12,Grimm Super You,2.0,-1.5,Grimm Artisanal Ales,660ml bottle - thanks Marduk! (4:2:4:2:8=2.0)\...,FatPhil
13,Magic Rock Dark Arts Hazelnut,1.9,-1.6,Magic Rock Brewing (Lion Co. - Kirin Holdings),"Lovely aromas of sweet chocolate, nuts and van...",Cool_Bond


In [28]:
# df.to_csv('all_reviews.csv',index=False)
# df = pd.read_csv('all_reviews.csv')

## Get more information on all of the beers - Selenium

In [10]:
from selenium import webdriver
import time

In [32]:
# extract already scraped data from previous scrapes so don't rescrape
beer_df = pd.read_csv('beer_deets.csv')

In [33]:
# compare to the unique list to create a new to-be-scraped list
new_beer = df.beer_name[~df.beer_name.isin(beer_df.name_found)].unique()
print("There are", len(new_beer), "new beers to search for!")

There are 1965 new beers to search for!


In [31]:
item_names = []
desc_string = []

driver = webdriver.Chrome(executable_path='/Users/lukebetham/Downloads/chromedriver')

for beer in tqdm(new_beer):
    driver.get("https://www.ratebeer.com/search?beername={}".format(beer))
    time.sleep(2)
    item_tags = driver.find_elements_by_class_name('fg-1')

    x=1
    beer_temp = []
    for item in item_tags:        
        if x <2:
            try:
                item_names.append(item.find_element_by_class_name('fd-r').text)
                x+=1
            except:
                pass
            try:
                beer_temp.append(item.text)
            except:
                pass
    try:
        desc_string.append(beer_temp[4])
    except:
        desc_string.append(None)

In [70]:
beer_deets = pd.DataFrame(desc_string, columns = ['return_string'])
beer_deets

,return_string
0,Flying Dog Raging Bitch Belgian-Style IPA\n🇺🇸 ...
1,Stone R & R Coconut IPA\n🇺🇸 IPA - Flavored • 7...
2,Yaletown Yippie IPA\n🇨🇦 IPA0\n51\n39\n3.32\n(2...
3,Yaletown Downtown Brown\n🇨🇦 Brown Ale0\n25\n23...
4,Yaletown Red Truck Ale\n🇨🇦 Red Ale / Intl Ambe...
...,...
1960,Fürst Carl Franken Ale\n🇩🇪 Blonde Ale / Golden...
1961,Brix City Parrot Jams\n🇺🇸 IIPA DIPA - Hazy / D...
1962,Lone Pine Quantum Cuddle Kitten\n🇺🇸 IIPA DIPA ...
1963,Munich Brew Mafia / Tap House Munich Anniversa...


In [71]:
## cleaning the scraped data
beer_deets.dropna(inplace=True)
beer_deets = beer_deets[~beer_deets.return_string.str.startswith('Logo\n')].copy()
no_alcohol = beer_deets[~beer_deets.return_string.str.contains('%')].copy()
beer_deets = beer_deets[beer_deets.return_string.str.contains('%')].copy()
# maybe create a cant find list so dont keep re-searching?

In [72]:
# sorting the scraped data into columns
beer_deets['name_found'] = beer_deets.return_string.apply(lambda x: x.split('\n')[0].strip())
beer_deets['flag'] = beer_deets.return_string.apply(lambda x: x.split('\n')[1].split('•')[0].split(" ")[0])
beer_deets['type'] = beer_deets.return_string.apply(lambda x: x.split('\n')[1].split('•')[0][2:])
beer_deets['abv'] = beer_deets.return_string.apply(lambda x: float(x.split('\n')[1].split('•')[1].replace("%","")))
beer_deets['rb_overall_score'] = beer_deets.return_string.apply(lambda x: x.split('\n')[2])
beer_deets['rb_style_score'] = beer_deets.return_string.apply(lambda x: x.split('\n')[3])
beer_deets['rb_user_rating'] = beer_deets.return_string.apply(lambda x: x.split('\n')[4])
beer_deets['rb_rating_number'] = beer_deets.return_string.apply(lambda x: x.split('\n')[5].replace("(","").replace(")",""))


In [73]:
# sorting the no alcohol measure
no_alcohol['name_found'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[0].strip())
no_alcohol['flag'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[1].split('•')[0].split(" ")[0])
no_alcohol['type'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[1].split('•')[0][2:])
no_alcohol['abv'] = no_alcohol.type.apply(lambda x: 0 if x[-1] == '0' else None)
no_alcohol['type'] = no_alcohol.type.apply(lambda x: x[:-1] if x[-1] == '0' else x)
no_alcohol['check'] = no_alcohol.return_string.apply(lambda x: len(x.split('\n')))
no_alcohol = no_alcohol[no_alcohol['check']>4].copy()
no_alcohol['rb_overall_score'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[2])
no_alcohol['rb_style_score'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[3])
no_alcohol['rb_user_rating'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[4])
no_alcohol['rb_rating_number'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[5].replace("(","").replace(")",""))
no_alcohol.drop('check',inplace=True, axis=1)

no_alcohol

,return_string,name_found,flag,type,abv,rb_overall_score,rb_style_score,rb_user_rating,rb_rating_number
2,Yaletown Yippie IPA\n🇨🇦 IPA0\n51\n39\n3.32\n(2...,Yaletown Yippie IPA,🇨🇦,IPA,0.0,51,39,3.32,24
3,Yaletown Downtown Brown\n🇨🇦 Brown Ale0\n25\n23...,Yaletown Downtown Brown,🇨🇦,Brown Ale,0.0,25,23,2.90,31
4,Yaletown Red Truck Ale\n🇨🇦 Red Ale / Intl Ambe...,Yaletown Red Truck Ale,🇨🇦,Red Ale / Intl Amber Ale,0.0,26,34,2.92,25
5,Yaletown Hills Special Wheat\n🇨🇦 Weissbier - H...,Yaletown Hills Special Wheat,🇨🇦,Weissbier - Hefeweizen,0.0,45,82,3.20,33
9,Nightmare Gunga Rao\n🇺🇸 IPA - Sour / Wild\n-\n...,Nightmare Gunga Rao,🇺🇸,IPA - Sour / Wild,NaN,-,-,3.67,3
...,...,...,...,...,...,...,...,...,...
1818,De La Vega's Pecan Grill Peanut Butter Pecan P...,De La Vega's Pecan Grill Peanut Butter Pecan P...,🇺🇸,Porter - Flavored,NaN,-,-,3.50,1
1852,Rainbow Olut Alkoholiton Lager\n🇧🇪 Low / No Al...,Rainbow Olut Alkoholiton Lager,🇧🇪,Low / No Alcohol Beer - Pale,0.0,0,3,1.23,41
1871,Cycle Sunday - 2020\n🇺🇸 BarleyWine / WheatWine...,Cycle Sunday - 2020,🇺🇸,BarleyWine / WheatWine / RyeWine,NaN,-,-,3.70,1
1894,Pyramid Subliminal State Hazy Pale Ale\n🇺🇸 Pal...,Pyramid Subliminal State Hazy Pale Ale,🇺🇸,Pale Ale - American / APA,NaN,-,-,3.70,1


In [74]:
beer_df = pd.concat([beer_df,beer_deets,no_alcohol])

In [75]:
beer_df

,return_string,flag,type,abv,rb_overall_score,rb_style_score,rb_user_rating,rb_rating_number,name_found
0,St. Peter's Ruby Red Ale\n🇬🇧 Bitter - Ordinary...,🇬🇧,Bitter - Ordinary / Best,4.3,49,95,3.25,415,St. Peter's Ruby Red Ale
1,Shepherd Neame Spitfire (Bottle)\n🇬🇧 Bitter - ...,🇬🇧,Bitter - Ordinary / Best,4.5,34,59,3.05,1083,Shepherd Neame Spitfire (Bottle)
2,Arbor Rocketman\n🇬🇧 IPA • 6.0%\n95\n97\n3.74\n...,🇬🇧,IPA,6.0,95,97,3.74,59,Arbor Rocketman
3,Shepherd Neame India Pale Ale (Bottle)\n🇬🇧 IPA...,🇬🇧,IPA - English,6.1,46,47,3.22,420,Shepherd Neame India Pale Ale (Bottle)
4,Shepherd Neame Christmas Ale\n🇬🇧 Strong Ale - ...,🇬🇧,Strong Ale - English,7.0,45,46,3.18,409,Shepherd Neame Christmas Ale
...,...,...,...,...,...,...,...,...,...
1818,De La Vega's Pecan Grill Peanut Butter Pecan P...,🇺🇸,Porter - Flavored,NaN,-,-,3.50,1,De La Vega's Pecan Grill Peanut Butter Pecan P...
1852,Rainbow Olut Alkoholiton Lager\n🇧🇪 Low / No Al...,🇧🇪,Low / No Alcohol Beer - Pale,0.0,0,3,1.23,41,Rainbow Olut Alkoholiton Lager
1871,Cycle Sunday - 2020\n🇺🇸 BarleyWine / WheatWine...,🇺🇸,BarleyWine / WheatWine / RyeWine,NaN,-,-,3.70,1,Cycle Sunday - 2020
1894,Pyramid Subliminal State Hazy Pale Ale\n🇺🇸 Pal...,🇺🇸,Pale Ale - American / APA,NaN,-,-,3.70,1,Pyramid Subliminal State Hazy Pale Ale


In [ ]:
# compare to the unique list to create a new to-be-scraped list
new_beer = df.beer_name[~df.beer_name.isin(beer_df.name_found)].unique()
print("There are", len(new_beer), "new beers to search for!")

In [22]:
# beer_df.to_csv('beer_deets.csv',index=False)